# Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
import sys
print(sys.executable)

c:\Users\singhp\Personal\Pranjay-Singh-Repo\architecture-projects\crew-ai\.venv\Scripts\python.exe


- Import from the crewAI libray.

In [3]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using Ollama running locally.

**Important Setup Steps:**
1. **Start Ollama**: Make sure Ollama is running with `ollama serve`
2. **Install a model**: Run `ollama pull llama3.2` (or `llama3`, `codellama`, etc.)
3. **Check available models**: Run `ollama list` to see installed models
4. **Verify connection**: Visit http://localhost:11434 in your browser

**Common models to try:**
- `llama3.2` (recommended)
- `llama3`
- `codellama`
- `mistral`

In [4]:
# Optional: Check what models are available in your Ollama installation
import requests
import json

def check_ollama_models():
    try:
        response = requests.get("http://localhost:11434/api/tags")
        if response.status_code == 200:
            models = response.json()
            print("🎯 Available Ollama models:")
            for model in models.get('models', []):
                print(f"  - {model['name']}")
            return [model['name'] for model in models.get('models', [])]
        else:
            print("❌ Could not connect to Ollama. Make sure it's running with 'ollama serve'")
            return []
    except Exception as e:
        print(f"❌ Error checking Ollama models: {e}")
        print("💡 Make sure Ollama is running: 'ollama serve'")
        return []

# Uncomment the line below to check available models
# available_models = check_ollama_models()

In [17]:
import os
from langchain_community.llms import Ollama

# Configure Ollama
os.environ["OPENAI_API_BASE"] = "http://localhost:11434/v1"
os.environ["OPENAI_MODEL_NAME"] = "mistral"  # Updated to a more common model
os.environ["OPENAI_API_KEY"] = "ollama"  # Dummy key for Ollama

# Initialize Ollama LLM - try different models if one doesn't work
try:
    # Try mistral first (most common)
    llm = Ollama(model="mistral", base_url="http://localhost:11434")
    print("✅ Successfully connected to Ollama with mistral")
except Exception as e:
    try:
        # Try llama3 as fallback
        llm = Ollama(model="mistral", base_url="http://localhost:11434")
        print("✅ Successfully connected to Ollama with llama3")
    except Exception as e:
        try:
            # Try codellama as another fallback
            llm = Ollama(model="codellama", base_url="http://localhost:11434")
            print("✅ Successfully connected to Ollama with codellama")
        except Exception as e:
            print(f"❌ Error connecting to Ollama: {e}")
            print("\n🔧 Troubleshooting steps:")
            print("1. Make sure Ollama is running: 'ollama serve'")
            print("2. Pull a model: 'ollama pull llama3.2' or 'ollama pull llama3'")
            print("3. List available models: 'ollama list'")
            print("4. Check if Ollama is accessible: http://localhost:11434")

✅ Successfully connected to Ollama with mistral


## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [18]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

### Agent: Writer

In [19]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

### Agent: Editor

In [20]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [21]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [22]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [23]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [24]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

2025-08-29 14:30:33,882 - 36452 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [25]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
 I now can give a great answer.

Final Answer:

Title: Navigating the Future: Unraveling Latest Trends, Key Players, and Developments in Artificial Intelligence (AI)

1. Introduction
   - Briefly explain what AI is and its relevance in today's digital age.
   - Grab reader attention by presenting an intriguing statistic or fact about AI's impact on various industries.
   - Introduce the purpose of the article: to help readers understand the latest trends, key players, and noteworthy news in AI.

2. Latest Trends in Artificial Intelligence
   - Explain cu

- Display the results of your execution as markdown in the notebook.

In [26]:
from IPython.display import Markdown
Markdown(result)

```markdown
Title: Navigating the Future: Unraveling Latest Trends, Key Players, and Developments in Artificial Intelligence (AI)

In today's digital landscape, Artificial Intelligence (AI) has become an integral part of our daily lives, shaping industries and transforming the way we interact with technology. This informative and engaging blog post aims to shed light on the latest trends, key players, and noteworthy news in AI, empowering readers to make informed decisions about this rapidly evolving field while keeping a neutral perspective.

**Latest Trends in Artificial Intelligence**

The world of AI is brimming with advancements in Machine Learning (ML), Deep Learning, Natural Language Processing (NLP), and Robotics. The rise of Edge AI, processing data closer to the source for improved efficiency, is gaining traction. Moreover, there's growing interest in Explainable AI, addressing concerns about transparency and accountability in AI systems. Lastly, AI ethics has emerged as a trending topic, emphasizing the importance of responsible AI development and deployment in maintaining trust with users.

**Key Players in the Artificial Intelligence Landscape**

Prominent tech giants like Google, Microsoft, IBM, and Amazon Web Services (AWS) are leading the charge in AI innovation. We strive to maintain an objective perspective by acknowledging their contributions while also highlighting emerging startups making waves in specific AI niches. Collaborations between academia, governments, and the private sector are vital for advancing AI research and applications.

**Noteworthy News and Upcoming Developments**

Recent breakthroughs, partnerships, acquisitions, or regulatory changes in the AI industry paint an exciting picture of its future. This blog post provides updates on these events without expressing personal opinions to offer a balanced viewpoint. Upcoming conferences, events, or publications related to AI offer valuable insights for those eager to learn more about this dynamic field.

**The Impact of Artificial Intelligence on Various Industries**

AI is being harnessed across sectors such as healthcare, finance, retail, and manufacturing. Each industry faces unique benefits and challenges when implementing AI solutions. The blog post presents these insights without taking a biased stance, allowing readers to draw their own conclusions.

**Addressing Audience Interests and Pain Points**

For businesses looking to adopt AI technology, practical tips are offered. These suggestions aim to help them make informed decisions while considering the potential impact on their operations. Resources for individuals interested in pursuing a career in AI or upskilling their current skillset are highlighted, providing opportunities for personal growth without promoting one option over another. Job displacement concerns caused by AI are addressed, offering reassurance and solutions such as reskilling initiatives and education programs that focus on the future of work.

**Call to Action**

We encourage you to share this article with others who might benefit from learning more about AI. Join the conversation by commenting on your thoughts and questions about the latest trends in AI, fostering a dialogue among our community. For further reading or viewing resources on AI-related topics, check out reputable sources such as MIT Technology Review, Forbes, or The Economist, and don't forget to explore Medium's Artificial Intelligence publication and VentureBeat's AI channel. We strive to maintain an objective perspective in our recommendations, ensuring a balanced viewpoint for our readers.
```

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "Cardio vs strength training"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Cardio vs strength training.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
 I now can give a great answer

Final Answer:

Content Plan Document for "Cardio vs Strength Training"

1. Target Audience Analysis:
   - Our primary target audience consists of individuals aged 18-50 who are interested in fitness, health, and wellness. They may have some knowledge of exercise but seek to improve their understanding of the benefits and tradeoffs between cardio and strength training. Our secondary audience includes fitness enthusiasts looking to optimize their workout routines and personal trainers seeking professional development.
  

In [ ]:
Markdown(result)

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).